In [1]:
import torch
import math
import torch.nn  as nn
import torch.nn.functional as F

In [29]:
input =torch.randn(5,5)
kernel =torch.randn(3,3)   

In [30]:
def  matrix_muti_for_cov(x,kernel,stride=1,padding=0):
    # kernel.shape ->(h,w)
    output_h= int((x.shape[0]-kernel.shape[0])/stride) +1
    output_w= int((x.shape[1]-kernel.shape[1])/stride) +1
    output =torch.zeros(output_h,output_w)
    
    for i in range (0,x.shape[0]-kernel.shape[0]+1,stride):
        
        for j in range (0,x.shape[1]-kernel.shape[1]+1,stride):
            
            area = x[i:i+kernel.shape[0],j:j+kernel.shape[1]]
            output[i,j] =torch.sum(area*kernel)
    return  output


output =matrix_muti_for_cov(input,kernel)
print(output)

tensor([[-2.0837, -1.1043,  3.2571],
        [-1.1638,  0.7576,  3.2776],
        [ 0.3669,  0.4015,  0.9808]])


In [31]:
input =input.reshape((1,1,input.shape[0],input.shape[1]))
kernel =kernel.reshape((1,1,kernel.shape[0],kernel.shape[1]))
cov_out =F.conv2d(input,kernel)
cov_out.squeeze(0).squeeze(0)

tensor([[-2.0837, -1.1043,  3.2571],
        [-1.1638,  0.7576,  3.2776],
        [ 0.3669,  0.4015,  0.9808]])

In [24]:
input =torch.randn(2,2,5,5)
kernel=torch.randn(2,2,3,3)
def  matrix_muti_for_cov2(input,kernel,bais =None,stride=1,padding=1):
    
    batch,channel,x_h,x_w =input.shape
    channel_out,channel_in,kernel_h,kernel_w =kernel.shape
    output_h= (math.floor((x_h+2*padding-kernel_h)/stride) +1)
    output_w= (math.floor((x_w+2*padding-kernel_w)/stride) +1)
    output =torch.zeros(batch,channel_out,output_h,output_w)
    if padding>0 :
        # input_padded = torch.zeros(batch, channel, x_h+2*padding, x_w+2*padding)
        # input_padded[:,:,padding:x_h+padding,padding:x_w+padding] =input
        input = torch.nn.functional.pad(input,[padding,padding,padding,padding],"constant", 0)
    if bais  is None:
        bais =torch.zeros(channel_out)
    for  b in range(batch):
        for c_out  in  range(channel_out):
                for i in range (0,output_h,stride):
                    for j in range (0,output_w,stride):
                        for c_in in range(channel_in):
                            area = input[b,c_in,i:i+kernel_h,j:j+kernel_w]
                            output[b,c_out,i,j] +=torch.sum(area*kernel[c_out,c_in])
                        output[b, c_out, i, j] += bais[c_out]            
                          
    return output
        

cov_out =matrix_muti_for_cov2(input,kernel)
# print(cov_out)
cov_out2 =F.conv2d(input,kernel,padding=1)
# print(cov_out2)
# if torch.allclose(cov_out, cov_out2, rtol=1e-05, atol=1e-08):
#     print("两个卷积结果近似相等。") 
# else:
#     print("两个卷积结果不相等。")  
#     print("最大绝对误差:", torch.max(torch.abs(cov_out - cov_out2))) 

flag = torch.allclose(cov_out,cov_out2)
print(flag)

True


In [66]:
input =torch.randn(5,5)
kernel =torch.randn(3,3) 
def  matrix_muti_for_cov(x,kernel,stride=1,padding=0):
    # kernel.shape ->(h,w)
    output_h= int((x.shape[0]-kernel.shape[0])/stride) +1
    output_w= int((x.shape[1]-kernel.shape[1])/stride) +1
    output =torch.zeros(output_h,output_w)
    area_matrix = torch.zeros(output.numel(),kernel.numel())
    kernel_matrix =kernel.reshape(kernel.numel(),-1)
    for i in range (0,x.shape[0]-kernel.shape[0]+1,stride):
        for j in range (0,x.shape[1]-kernel.shape[1]+1,stride):
            
            area = x[i:i+kernel.shape[0],j:j+kernel.shape[1]]
            area_matrix[i+j] = torch.flatten(area)
    output_matrix =area_matrix@ kernel_matrix
    output = output_matrix.reshape(output_h, output_w)
    return  output